In [17]:
!poetry add catboost

zsh:1: command not found: poetry


In [18]:
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier, Pool
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import joblib
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import cross_val_score, HalvingGridSearchCV
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_score, recall_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import warnings
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import make_column_selector, make_column_transformer


warnings.filterwarnings('ignore')

In [19]:
def pandas_preprocessing(X, y_present=True):
  X['PassengerId_split'] = X['PassengerId'].apply(lambda x : str(x).split(sep = '_', maxsplit=1))
  X['Group']= X['PassengerId_split'].apply(lambda x : np.nan if x[0] == 'nan' else x[0])
  X['Group_id']= X['PassengerId_split'].apply(lambda x : np.nan if x[0] == 'nan' else x[1])
  X = X.drop(['PassengerId','PassengerId_split'],axis=1)

  # Split the names
  X['Name_split'] = X['Name'].apply(lambda x : str(x).split(sep = ' ', maxsplit=1))
  X['Surname']= X['Name_split'].apply(lambda x : np.nan if x[0] == 'nan' else x[1])
  X = X.drop(['Name','Name_split'],axis=1)

  # Split the cabins
  X['Cabin_splt'] = X['Cabin'].apply(lambda x : str(x).split(sep = '/'))
  X['Deck']= X['Cabin_splt'].apply(lambda x : np.nan if x[0] == 'nan' else x[0])
  X['CabinNumber']= X['Cabin_splt'].apply(lambda x : np.nan if x[0] == 'nan' else x[1]).astype('float')
  X['CabinSide']= X['Cabin_splt'].apply(lambda x : np.nan if x[0] == 'nan' else x[2])
  X = X.drop(['Cabin','Cabin_splt'],axis=1)

  # Create Total
  numeric_cols = list(X.select_dtypes(include = np.number).columns)
  X['Total'] = sum(X[col] for col in numeric_cols if col not in ["Age", "CabinNumber"])
  # X['age'] = pd.cut(x=X['Age'], bins=range(0, 90, 10))
  # X['total'] = pd.cut(x=X['Total'], bins=[-35.987, 3598.7, 7197.4, 10796.1, 14394.8, 17993.5, 21592.2, 25190.9, 28789.6, 32388.3, 35987.])
  # X = X.drop(['Age', "Total"],axis=1)
  # X['age'] = X['age'].astype('str') 
  # X['total'] = X['total'].astype('str')
  target = X['Transported']
  target = target.astype(int)
  X = X.drop(['Transported'],axis=1)
  return X, target

In [20]:
bol_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    )

cat_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
    )


num_pipeline = make_pipeline(
    SimpleImputer(strategy='median'),
    StandardScaler()
)

preprocessing = make_column_transformer(
    (num_pipeline, make_column_selector(dtype_include=np.number)),
    (cat_pipeline, make_column_selector(dtype_include=object)),
    (bol_pipeline, make_column_selector(dtype_include=bool)),
)

In [21]:
%matplotlib inline

In [22]:
df = pd.read_csv('data_FAA/train.csv')
df, y = pandas_preprocessing(df)
df_prepared = preprocessing.fit_transform(df)

In [23]:
df_prepared_fr = pd.DataFrame(
    df_prepared,
    columns=preprocessing.get_feature_names_out(),
    index=df.index)
df_prepared_fr.head()
X_train, X_test, y_train, y_test = train_test_split(df_prepared_fr, y, test_size = 0.2, random_state = 100)

In [24]:
import joblib
with open('data_FAA/data2.pkl', 'wb') as f:
    joblib.dump([X_train, y_train, X_test, y_test], f)

In [25]:
X_train, y_train, X_test, y_test = joblib.load('data_FAA/data2.pkl')
redo_grid_search = 1
cm = 0

In [26]:
if redo_grid_search:
  pool_train = Pool(
      X_train,
      y_train
  )

  pool_test = Pool(
      X_test,
      y_test
  )

  search_parameters = {
      "learning_rate": [0.01, 0.1, 0.5],
      "iterations": [400, 700, 1000],
      "depth": [4, 6, 10],
      "l2_leaf_reg": [1, 3, 5]
  }

  cb_classifier = CatBoostClassifier(random_state=0xDEADBEEF, logging_level="Silent")

  best_params = cb_classifier.grid_search(
      search_parameters,
      X=pool_train,
      cv=3,
      verbose=15,
  #     plot=True
  )

  print(f"Best parameters:\n{best_params['params']}")
  print(f"Best score: {cb_classifier.get_best_score()}")
  cb_best = best_params.best_estimator_
else:
  print("cneas")

0:	loss: 0.4357160	best: 0.4357160 (0)	total: 822ms	remaining: 1m 5s
15:	loss: 0.4132051	best: 0.3804545 (13)	total: 43.4s	remaining: 2m 56s
30:	loss: 0.4136277	best: 0.3804545 (13)	total: 1m 19s	remaining: 2m 7s
45:	loss: 0.3859772	best: 0.3798327 (31)	total: 1m 57s	remaining: 1m 29s
60:	loss: 0.3999904	best: 0.3798327 (31)	total: 3m 17s	remaining: 1m 4s
75:	loss: 0.3840018	best: 0.3798327 (31)	total: 6m 39s	remaining: 26.3s


In [ ]:
y_pred = cross_val_predict(cb_best, X_train, y_train, cv=20)
acc = accuracy_score(y_train, y_pred)
print(f"The accuracy score for this model is {acc}")

In [ ]:
from sklearn.metrics import RocCurveDisplay
cb_best.fit(X_train, y_train)
RocCurveDisplay.from_estimator(
   cb_best, X_test, y_test)


In [ ]:
from sklearn.metrics import PrecisionRecallDisplay
PrecisionRecallDisplay.from_estimator(
   cb_best, X_test, y_test)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(2, 2, figsize=(12, 8))
RocCurveDisplay.from_estimator(
   cb_best, X_test, y_test, ax=ax[1, 1])


PrecisionRecallDisplay.from_estimator(
   cb_best, X_test, y_test, ax=ax[1, 0])

ConfusionMatrixDisplay.from_predictions(y_train, y_pred, ax=ax[0, 0])
ConfusionMatrixDisplay.from_predictions(y_train, y_pred, normalize="true", ax=ax[0, 1])
plt.show()

In [ ]:
%%time

# always use same RANDOM_STATE k-folds for comparability between tests, reproducibility
RANDOMSTATE = 42
np.random.seed(RANDOMSTATE)

kfolds = KFold(n_splits=20, shuffle=True, random_state=RANDOMSTATE)

MEAN_RESPONSE=y_train.mean()
def cv_to_raw(cv_val, mean_response=MEAN_RESPONSE):
    return np.expm1(mean_response+cv_val) - np.expm1(mean_response)
	
scores = -cross_val_score(cb_best, X_train, y_train,
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds,
                          n_jobs=-1)
raw_scores = [cv_to_raw(x) for x in scores]
print("Raw CV RMSE %.0f (STD %.0f)" % (np.mean(raw_scores), np.std(raw_scores)))

In [ ]:
for score, name in zip(cb_best.feature_importances_, X_train.columns):
    print(f"{score:.3f}{' ':>20}{name.split('__', 1)[1]:>20}")

In [ ]:
train_sizes, train_scores, valid_scores = learning_curve(
    cb_best, X_train, y_train, train_sizes=np.linspace(0.01, 1.0, 40), cv=5,
    scoring="neg_root_mean_squared_error")
train_errors = -train_scores.mean(axis=1)
valid_errors = -valid_scores.mean(axis=1)

plt.figure(figsize=(6, 4))  # extra code – not needed, just formatting
plt.plot(train_sizes, train_errors, "r-+", linewidth=2, label="train")
plt.plot(train_sizes, valid_errors, "b-", linewidth=3, label="valid")

# extra code – beautifies and saves Figure 4–15
plt.xlabel("Training set size")
plt.ylabel("RMSE")
plt.grid()
plt.legend(loc="upper right")
# plt.axis([0, 1000, 0.3, 0.9])
# plt.save_fig("underfitting_learning_curves_plot")

plt.show()

In [ ]:
#----------------For submission----------------
data1= pd.read_csv("data_FAA/train.csv")
data2 = pd.read_csv("data_FAA/evaluation.csv")
data2 = pd.read_csv("data_FAA/evaluation.csv")
data2['Transported'] =-1

data1, y1 = pandas_preprocessing(data1)
data2, y2 = pandas_preprocessing(data2)

data1 = preprocessing.fit_transform(data1)
data2 = preprocessing.transform(data2)

cb_best.fit(data1, y1)

pred = cb_best.predict(data2)

pred = pred.astype(bool)
result=pd.read_csv('data_FAA/sample_submission.csv')
result['Transported'] = pred
result.to_csv('submission_cat.csv', index=False)